In [51]:
import pandas as pd
import numpy as np
from numpy import nan

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [52]:
url = "https://www.windguru.cz/208690"

In [72]:
Xdates = '//*[@id="tabid_0_0_dates"]'
Xwind = '//*[@id="tabid_0_0_WINDSPD"]'
Xgusts = '//*[@id="tabid_0_0_GUST"]'
XwindDirection = '//*[@id="tabid_0_0_SMER"]'
Xswell = '//*[@id="tabid_0_0_HTSGW"]'
Xperiod = '//*[@id="tabid_0_0_PERPW"]'
XswellDirection = '//*[@id="tabid_0_0_DIRPW"]'

options = Options()
options.add_argument("--headless=new")
service = webdriver.ChromeService()
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)

try:
    dates = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, Xdates))).text
    #print(dates)
    wind = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, Xwind))).text
    #print(wind)
    gusts = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, Xgusts))).text
    #print(gusts)
    wind_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, XwindDirection)))
    span_elements = WebDriverWait(wind_element, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/span[@title]')))
    windDirections = []
    for span_element in span_elements:
        span_title = span_element.get_attribute('title')
        windDirections.append(span_title)
    #print(windDirection)
    swell = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, Xswell))).text
    #print(swell)
    period = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, Xperiod))).text
    #print(period)
    
    swell_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, XswellDirection)))
    span_elements = WebDriverWait(swell_element, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/span[@title]')))
    swellDirections = []
    for span_element in span_elements:
        span_title = span_element.get_attribute('title')
        swellDirections.append(span_title)
    #print(windDirection)
except:
        print("Something failed")
driver.quit()

In [54]:
dates = dates.split('\n')
dates = dates[1:]

In [55]:
datesList = []
counter = 0
for date in dates:
    if counter%2 != 0:
        datesList.append(date + " " + dates[counter-1])
    counter += 1

In [56]:
index = ['wind', 'gusts', 'windDirection', 'swell', 'period', 'swellDirection']

In [57]:
df = pd.DataFrame(columns = datesList, index=index)

In [58]:
windList = wind.split()

In [59]:
df.loc['wind'] = windList

In [60]:
gustsList = gusts.split()

In [61]:
df.loc['gusts'] = gustsList

In [62]:
df.loc['windDirection'] = windDirections

In [63]:
swell = swell.split()
df.loc['swell'] = swell

In [64]:
period = period.split()
df.loc['period'] = period

In [65]:
df.loc['swellDirection'] = swellDirections

In [66]:
df

,11h Mo 4.,13h Mo 4.,15h Mo 4.,17h Mo 4.,19h Mo 4.,21h Tu 4.,03h Tu 5.,05h Tu 5.,07h Tu 5.,09h Tu 5.,...,11h Mo 18.,14h Mo 18.,17h Mo 18.,20h Tu 18.,05h Tu 19.,08h Tu 19.,11h Tu 19.,14h Tu 19.,17h Tu 19.,20h 19.
wind,14,12,11,13,11,9,3,3,4,4,...,13,15,15,14,11,8,7,8,7,7
gusts,16,14,12,14,12,13,5,4,5,5,...,16,17,16,17,14,10,7,6,7,10
windDirection,S (175°),SSE (166°),SSE (167°),SSE (156°),SE (146°),SE (145°),SSE (158°),SSW (194°),S (179°),SSE (152°),...,SSW (192°),SSE (167°),SSE (148°),SSE (155°),SSE (165°),S (176°),SSE (167°),SE (125°),ESE (108°),ENE (73°)
swell,3.3,3.2,3.1,3.1,3.1,3,2.6,2.5,2.5,2.4,...,1.7,1.9,2.2,2.3,2.9,2.8,2.6,2.4,2.2,2
period,10,10,10,15,14,14,13,13,13,13,...,10,6,7,8,10,10,12,13,13,12
swellDirection,S (178°),S (177°),S (177°),S (173°),S (172°),S (172°),S (171°),S (170°),S (170°),S (170°),...,S (183°),S (176°),S (169°),SSE (167°),S (174°),S (174°),S (174°),S (173°),S (173°),S (172°)


In [67]:
df.loc['wind']=df.loc['wind'].astype(int)
df.loc['gusts']=df.loc['gusts'].astype(int)
df.loc['swell']=df.loc['swell'].astype(float)
df.loc['period']=df.loc['period'].astype(int)

In [68]:
df.loc['swell'][0]

3.3

In [70]:
print("possible days/times to look out for:")
for col_index, col in enumerate(df):
    #day = col.split()[1])
    if(df.loc['swell'][col_index] > 2.0 and (df.loc['windDirection'][col_index].split()[0] in {'W', 'NW', 'SW', 'NNW', 'SSW', 'WNW', 'WSW'})):
        print(col)

possible days/times to look out for:
05h Tu 5.
